# Sentiment Analysis Using NLP

In [2]:
import pandas as pd
import nltk

In [6]:
df = pd.read_csv("./Tweets.csv")
df.sample(2)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
5918,568489413262610433,neutral,1.0,NaN,NaN,Southwest,NaN,TTpKP12,NaN,0,@SouthwestAir do you have any flights from NAS...,NaN,2015-02-19 11:16:52 -0800,NaN,Central Time (US & Canada)
10109,569533202835984385,negative,1.0,Customer Service Issue,1.0,US Airways,NaN,68aec68,NaN,0,@USAirways I waited customer service 32 minute...,NaN,2015-02-22 08:24:31 -0800,NaN,Atlantic Time (Canada)


In [7]:
df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [9]:
#df.drop(columns=['tweet_id', 'airline_sentiment_confidence','negativereason', 'negativereason_confidence', 'airline','airline_sentiment_gold', 'name', 'negativereason_gold','retweet_count', 'tweet_coord', 'tweet_created','tweet_location', 'user_timezone'],inplace=True)
df = df[["airline_sentiment", "text"]]
df.head(2)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...


In [10]:
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
ps = PorterStemmer()


def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [11]:
df['text_cleaned'] = df['text'].apply(clean_text)
df.sample(3)

,airline_sentiment,text,text_cleaned
9694,negative,@USAirways you are absolutely terrible at mana...,@ usairway absolut terribl manag delay ... del...
5575,neutral,@SouthwestAir go @franks105 go!,@ southwestair go @ franks105 go !
7210,neutral,@JetBlue to start 3xweekly @EmbraerSA #E190 fl...,@ jetblu start 3xweekli @ embraersa # e190 fli...


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tf_vec = TfidfVectorizer(max_features=3000)
x = tf_vec.fit_transform(df['text_cleaned']).toarray()
x.shape

(14640, 3000)

In [15]:
y = df['airline_sentiment'].values

In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [17]:
from sklearn.naive_bayes import MultinomialNB
#naive_bayes ==> machine learning model that is being used

model = MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

MultinomialNB()

In [18]:
from sklearn.metrics import accuracy_score
y_prediction = model.predict(x_test)
print(accuracy_score(y_test,y_prediction))

0.7219945355191257


In [19]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

print(accuracy_score(y_test,y_pred))

0.7503415300546448


In [23]:
df = df.drop_duplicates(keep='first')
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9087   9087   
neutral            3067   3067   
positive           2298   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @VirginAmerica it's really aggressive to blast...    1   
neutral                          @VirginAmerica What @dhepburn said.    1   
positive           @VirginAmerica plus you've added commercials t...    1   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9087   9083   
neutral                   3067   3025   
positive                  2298   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ unit airlin cancel flight oc fll # flight to...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                              @ unit thank !    5